# Лямбда-исчисление: От теории к практике
## Интерактивное введение в функциональное программирование

Этот notebook проведёт вас через основы лямбда-исчисления и покажет, как эти идеи используются в современном программировании.

## Часть 1: Основы лямбда-исчисления

Лямбда-исчисление — минималистичная формальная система из трёх элементов:

1. **Переменные**: `x`, `y`, `z`
2. **Абстракция** (создание функции): `λx.M`
3. **Применение** (вызов функции): `(M N)`

В Python лямбда-выражения записываются так:
```
λx.x  →  lambda x: x
```

In [ ]:
# Простейшая функция — тождественная (identity)
identity = lambda x: x

print("Тест identity функции:")
print(identity(5))        # 5
print(identity("hello"))  # "hello"

### β-редукция — основное правило вычисления

```
(λx.x + 1) 5  →  5 + 1  →  6
```

Это правило подстановки: заменяем параметр на аргумент.

In [ ]:
add_one = lambda x: x + 1
print("Тест β-редукции:")
print(add_one(5))  # 6

## Часть 2: Каррирование и замыкания

**Каррирование**: функция с несколькими параметрами превращается в цепочку функций с одним параметром.

```
λx.λy.x + y
```

означает "функция, которая принимает `x` и возвращает функцию, которая принимает `y`"

In [ ]:
# Обычная функция vs каррированная
def add_normal(x, y):
    return x + y

add_curried = lambda x: lambda y: x + y

print("=== Каррирование ===")
print("Обычная функция:", add_normal(3, 5))

# Каррированная функция может быть частично применена
add_3 = add_curried(3)  # Замыкание! Помнит x=3
print("Частичное применение:", add_3(5))
print("Полное применение:", add_curried(3)(5))

### Практический пример: создание специализированных функций

In [ ]:
multiply = lambda x: lambda y: x * y

double = multiply(2)
triple = multiply(3)
times_10 = multiply(10)

numbers = [1, 2, 3, 4, 5]
print("Практическое применение:")
print("Удвоение:", list(map(double, numbers)))
print("Утроение:", list(map(triple, numbers)))

## Часть 3: Числа Чёрча

Алонзо Чёрч показал, что числа можно закодировать через функции!

**Идея**: число n = "применить функцию n раз"

```
0 = λf.λx.x           (не применяем f)
1 = λf.λx.f x         (применяем f один раз)  
2 = λf.λx.f (f x)     (применяем f два раза)
3 = λf.λx.f (f (f x)) (применяем f три раза)
```

In [ ]:
# Определяем числа Чёрча
zero  = lambda f: lambda x: x
one   = lambda f: lambda x: f(x)
two   = lambda f: lambda x: f(f(x))
three = lambda f: lambda x: f(f(f(x)))
four  = lambda f: lambda x: f(f(f(f(x))))
five  = lambda f: lambda x: f(f(f(f(f(x)))))

### Визуализация чисел Чёрча

In [ ]:
# Визуализация: добавляем звёздочки
add_star = lambda s: s + "*"

print("=== Числа Чёрча ===")
print(f"zero:  '{zero(add_star)('')}'")
print(f"one:   '{one(add_star)('')}'")
print(f"two:   '{two(add_star)('')}'")
print(f"three: '{three(add_star)('')}'")
print(f"four:  '{four(add_star)('')}'")
print(f"five:  '{five(add_star)('')}'")

In [ ]:
# Преобразование в обычные числа Python
def church_to_int(church_num):
    """Конвертирует число Чёрча в обычное целое число"""
    return church_num(lambda n: n + 1)(0)

print("Преобразование в int:")
print(f"three = {church_to_int(three)}")
print(f"five = {church_to_int(five)}")

## Часть 4: Арифметика с числами Чёрча

### Сложение

Если `m` применяет `f` m раз, а `n` применяет `f` n раз, то сначала применим `n`, потом `m` к результату → (m+n) раз

In [ ]:
def add(m, n):
    """Сложение чисел Чёрча"""
    return lambda f: lambda x: m(f)(n(f)(x))

print("=== Арифметика ===")
result = add(two, three)
print(f"two + three = {church_to_int(result)}")

result = add(four, one)
print(f"four + one = {church_to_int(result)}")

### Умножение

"Применить (применить `f` n раз) m раз" = применить `f` (m × n) раз

In [ ]:
def multiply(m, n):
    """Умножение чисел Чёрча"""
    return lambda f: m(n(f))

result = multiply(two, three)
print(f"two × three = {church_to_int(result)}")

result = multiply(three, four)
print(f"three × four = {church_to_int(result)}")

## Часть 5: Булевы значения

Булевы значения = функции-селекторы

```
true  = λx.λy.x  (выбирает первый аргумент)
false = λx.λy.y  (выбирает второй аргумент)
```

In [ ]:
true  = lambda x: lambda y: x
false = lambda x: lambda y: y

print("=== Булевы значения ===")
print(f"true выбирает:  {true('первый')('второй')}")
print(f"false выбирает: {false('первый')('второй')}")

### Условный оператор

In [ ]:
def if_then_else(condition, then_value, else_value):
    """
    condition — это уже функция-селектор!
    Просто передаём ей значения для выбора
    """
    return condition(then_value)(else_value)

print("Условный оператор:")
print(f"if true then 'да' else 'нет': {if_then_else(true, 'да', 'нет')}")
print(f"if false then 'да' else 'нет': {if_then_else(false, 'да', 'нет')}")

### Логические операции

In [ ]:
def and_op(a, b):
    """Логическое И через лямбда-исчисление"""
    return a(b)(false)

def or_op(a, b):
    """Логическое ИЛИ через лямбда-исчисление"""
    return a(true)(b)

def not_op(a):
    """Логическое НЕ через лямбда-исчисление"""
    return a(false)(true)

# Преобразование в Python bool
to_bool = lambda church_bool: church_bool(True)(False)

print("Логические операции:")
print(f"true AND true = {to_bool(and_op(true, true))}")
print(f"true AND false = {to_bool(and_op(true, false))}")
print(f"true OR false = {to_bool(or_op(true, false))}")
print(f"NOT true = {to_bool(not_op(true))}")

## Часть 6: Рекурсия без имён

**Проблема**: в чистом лямбда-исчислении нет имён функций. Как сделать рекурсию без имени?

**Решение**: функция получает саму себя как параметр!

In [ ]:
print("=== Рекурсия ===")

# Самоприменяющийся факториал
factorial_self = lambda self, n: 1 if n == 0 else n * self(self, n - 1)

print("Факториал через самоприменение:")
print(f"factorial(5) = {factorial_self(factorial_self, 5)}")
print(f"factorial(7) = {factorial_self(factorial_self, 7)}")

In [ ]:
# Числа Фибоначчи
fib_self = lambda self, n: n if n <= 1 else self(self, n-1) + self(self, n-2)

print("Числа Фибоначчи:")
for i in range(10):
    print(f"fib({i}) = {fib_self(fib_self, i)}", end="  ")
print()

## Часть 7: Y-комбинатор (продвинутый уровень)

Y-комбинатор — это "магическая" функция, которая добавляет рекурсию к любой функции без использования имён.

```
Y = λf.(λx.f(x x))(λx.f(x x))
```

Упрощённая версия для Python:

In [ ]:
def Y(f):
    """Упрощённый Y-комбинатор для Python"""
    return (lambda x: f(lambda *args: x(x)(*args)))(
            lambda x: f(lambda *args: x(x)(*args)))

# Теперь можем писать рекурсивные функции без самоссылок!
factorial = Y(lambda f: lambda n: 1 if n == 0 else n * f(n - 1))

print("=== Y-комбинатор ===")
print(f"factorial(6) = {factorial(6)}")

fibonacci = Y(lambda f: lambda n: n if n <= 1 else f(n-1) + f(n-2))
print(f"fibonacci(10) = {fibonacci(10)}")

## Часть 8: Практическое применение

Идеи из лямбда-исчисления используются в современном программировании:

### 1. Композиция функций

In [ ]:
def compose(f, g):
    """Композиция: (f ∘ g)(x) = f(g(x))"""
    return lambda x: f(g(x))

add_10 = lambda x: x + 10
multiply_by_2 = lambda x: x * 2

# Сначала умножаем, потом прибавляем
transform = compose(add_10, multiply_by_2)
print(f"compose(add_10, multiply_by_2)(5) = {transform(5)}")  # (5*2)+10 = 20

### 2. Частичное применение (partial application)

In [ ]:
def partial(f, *fixed_args):
    """Фиксирует первые аргументы функции"""
    return lambda *remaining_args: f(*fixed_args, *remaining_args)

def greet(greeting, name):
    return f"{greeting}, {name}!"

say_hello = partial(greet, "Привет")
say_goodbye = partial(greet, "До свидания")

print(say_hello('Алиса'))
print(say_goodbye('Боб'))

### 3. Функции высшего порядка

In [ ]:
from functools import reduce

numbers = [1, 2, 3, 4, 5]

# map, filter, reduce — всё из функционального программирования
squared = list(map(lambda x: x**2, numbers))
evens = list(filter(lambda x: x % 2 == 0, numbers))
sum_all = reduce(lambda acc, x: acc + x, numbers, 0)

print(f"Возведение в квадрат: {squared}")
print(f"Только чётные: {evens}")
print(f"Сумма всех: {sum_all}")

## Заключение

Лямбда-исчисление показало, что:

1. **Функции — универсальный строительный блок вычислений**
2. **Из простых правил можно построить сложные системы**
3. **Вычисления = композиция и применение функций**

Эти идеи лежат в основе:
- Функционального программирования (Haskell, Lisp, ML)
- Функций высшего порядка в современных языках
- Замыканий и лексического скоупа
- Иммутабельности и чистых функций
- Систем типов и type inference

**Когда вы пишете `.map()`, `.filter()`, используете замыкания или композицию — вы применяете принципы лямбда-исчисления!**

In [ ]:
print("="*50)
print("Поздравляю! Вы прошли путь от теории к практике.")
print("="*50)